# Serving Alibi-Detect models

Out of the box, `mlserver` supports the deployment and serving of `alibi-detect` models. In this example, we will cover how we can create a detector configuration to then serve it using `mlserver`.

## Reference Data and Configuration

The first step will be to fetch a reference data for an `alibi-detect` model. For that, we will use the [income Classifier example from the `alibi-detect` documentation](https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_chi2ks_adult.html)

In [1]:
import alibi
import matplotlib.pyplot as plt
import numpy as np

In [2]:
adult = alibi.datasets.fetch_adult()
X, y = adult.data, adult.target
feature_names = adult.feature_names
category_map = adult.category_map
X.shape, y.shape

((32561, 12), (32561,))

In [3]:
n_ref = 10000
n_test = 10000

X_ref, X_t0, X_t1 = X[:n_ref], X[n_ref:n_ref + n_test], X[n_ref + n_test:n_ref + 2 * n_test]
X_ref.shape, X_t0.shape, X_t1.shape

((10000, 12), (10000, 12), (10000, 12))

In [4]:
categories_per_feature = {f: None for f in list(category_map.keys())}

### Saving our reference data

In [5]:

detector_data={"x_ref":X_ref,"categories_per_feature":categories_per_feature}

In [6]:
import pickle
filepath = 'alibi-detector-artifacts/detector_data.pkl' 
pickle.dump(detector_data, open(filepath,"wb"))

## Serving

Now that we have the reference data and other configuration parameters, the next step will be to serve it using `mlserver`. 
For that, we will need to create 2 configuration files: 

- `settings.json`: holds the configuration of our server (e.g. ports, log level, etc.).
- `model-settings.json`: holds the configuration of our model (e.g. input type, runtime to use, etc.).

### `settings.json`

In [7]:
%%writefile settings.json
{
    "debug": "true"
}

Overwriting settings.json


### `model-settings.json`

In [17]:
%%writefile model-settings.json
{
  "name": "income-classifier-cd",
  "implementation": "mlserver_alibi_detect.AlibiDriftDetectRuntime",
  "parameters": {
    "uri": "./alibi-detector-artifacts/detector_data.pkl",
    "version": "v0.1.0",
    "extra":{
      "detector_type":"alibi_detect.cd.TabularDrift",
      "protocol": "kfserving.http",
      "init_detector": true,
      "init_parameters": {
        "p_val": 0.05
      },
      "predict_parameters": {
        "drift_type": "feature"
      }
    }
  }
}

Overwriting model-settings.json


### Start serving our model

Now that we have our config in-place, we can start the server by running `mlserver start .`. This needs to either be ran from the same directory where our config files are or pointing to the folder where they are.

```shell
mlserver start .
```

Since this command will start the server and block the terminal, waiting for requests, this will need to be ran in the background on a separate terminal.

### Send test inference request

We now have our model being served by `mlserver`.
To make sure that everything is working as expected, let's send a request from our test set.

For that, we can use the Python types that `mlserver` provides out of box, or we can build our request manually.

# TabularDrift

### Detecting Drift locally

In [9]:
from alibi_detect.cd import TabularDrift
cd = TabularDrift(X_ref, p_val=.05, categories_per_feature=categories_per_feature)
cd.predict(X_t0,drift_type="feature")

{'data': {'is_drift': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'distance': array([1.1600000e-02, 8.4865131e+00, 4.7529316e+00, 3.1599441e+00,
         8.1941366e+00, 4.8458526e-01, 5.8652312e-01, 2.1689056e-01,
         2.4000001e-03, 1.6000000e-03, 1.1600000e-02, 9.9910326e+00],
        dtype=float32),
  'p_val': array([0.50786483, 0.38744345, 0.5758693 , 0.36761624, 0.41473988,
         0.99267685, 0.9645494 , 0.6414195 , 1.        , 1.        ,
         0.50786483, 0.44128036], dtype=float32),
  'threshold': 0.05},
 'meta': {'name': 'TabularDrift',
  'detector_type': 'offline',
  'data_type': None}}

### Detecting Drift via MLServer

In [13]:
import requests

inference_request = {
    "inputs": [
        {
            "name": "predict",
            "shape": X_t0.shape,
            "datatype": "FP32",
            "data": X_t0.tolist(),
        }
    ],
}

endpoint = "http://localhost:8080/v2/models/income-classifier-cd/versions/v0.1.0/infer"
response = requests.post(endpoint, json=inference_request)

In [14]:
import json
response_dict = json.loads(response.text)
print(response_dict,"\n")

labels = ['No!', 'Yes!']
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = response_dict['outputs'][0]['data'][f]
    print(f'{fname} -- Drift? {labels[is_drift]}')

{'model_name': 'income-classifier-cd', 'model_version': 'v0.1.0', 'id': 'baf24e6b-99fe-41f6-9735-c905a39a8ca8', 'parameters': {'content_type': None, 'name': 'TabularDrift', 'data_type': None, 'detector_type': 'offline'}, 'outputs': [{'name': 'detect', 'shape': [12], 'datatype': 'INT64', 'parameters': None, 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}]} 

Age -- Drift? No!
Workclass -- Drift? No!
Education -- Drift? No!
Marital Status -- Drift? No!
Occupation -- Drift? No!
Relationship -- Drift? No!
Race -- Drift? No!
Sex -- Drift? No!
Capital Gain -- Drift? No!
Capital Loss -- Drift? No!
Hours per week -- Drift? No!
Country -- Drift? No!


### Detecting Drift via custom endpoint for v2 protocol

In [18]:
import requests

inference_request = {
    "inputs": [
        {
            "name": "predict",
            "shape": X_t0.shape,
            "datatype": "FP32",
            "data": X_t0.tolist(),
        }
    ],
}

endpoint = "http://localhost:8080/"
response = requests.post(endpoint, json=inference_request)

In [19]:
import json
response_dict = json.loads(response.text)
print(response_dict,"\n")

labels = ['No!', 'Yes!']
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = response_dict['data']['is_drift'][f]
    stat_val, p_val = response_dict['data']['distance'][f], response_dict['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

{'data': {'is_drift': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'distance': [0.0116, 8.486513, 4.7529316, 3.159944, 8.194137, 0.48458526, 0.5865231, 0.21689056, 0.0024, 0.0016, 0.0116, 9.991033], 'p_val': [0.50786483, 0.38744345, 0.5758693, 0.36761624, 0.41473988, 0.99267685, 0.9645494, 0.6414195, 1.0, 1.0, 0.50786483, 0.44128036], 'threshold': 0.05}, 'meta': {'name': 'TabularDrift', 'detector_type': 'offline', 'data_type': None}} 

Age -- Drift? No! -- K-S 0.012 -- p-value 0.508
Workclass -- Drift? No! -- Chi2 8.487 -- p-value 0.387
Education -- Drift? No! -- Chi2 4.753 -- p-value 0.576
Marital Status -- Drift? No! -- Chi2 3.160 -- p-value 0.368
Occupation -- Drift? No! -- Chi2 8.194 -- p-value 0.415
Relationship -- Drift? No! -- Chi2 0.485 -- p-value 0.993
Race -- Drift? No! -- Chi2 0.587 -- p-value 0.965
Sex -- Drift? No! -- Chi2 0.217 -- p-value 0.641
Capital Gain -- Drift? No! -- K-S 0.002 -- p-value 1.000
Capital Loss -- Drift? No! -- K-S 0.002 -- p-value 1.000
Hours per week -- Drif

### Detecting Drift via custom endpoint for Tensorflow protocol

In [25]:
%%writefile model-settings.json
{
  "name": "income-classifier-cd",
  "implementation": "mlserver_alibi_detect.AlibiDriftDetectRuntime",
  "parameters": {
    "uri": "./alibi-detector-artifacts/detector_data.pkl",
    "version": "v0.1.0",
    "extra":{
      "detector_type":"alibi_detect.cd.TabularDrift",
      "init_detector":true,
      "protocol": "tensorflow.http",
      "init_parameters": {
        "p_val": 0.05
      },
      "predict_parameters": {
        "drift_type": "feature"
      }
    }
  }
}

Overwriting model-settings.json


In [23]:
import requests

inference_request = {
    "instances": X_t1.tolist()
}

endpoint = "http://localhost:8080/"
response = requests.post(endpoint, json=inference_request)

In [24]:
import json
response_dict = json.loads(response.text)
print(response_dict,"\n")

labels = ['No!', 'Yes!']
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = response_dict['data']['is_drift'][f]
    stat_val, p_val = response_dict['data']['distance'][f], response_dict['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

{'data': {'is_drift': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'distance': [0.007, 5.799565, 5.4128075, 1.1667116, 12.295819, 6.519854, 1.4171993, 0.008140487, 0.0054, 0.0033, 0.0045, 11.255669], 'p_val': [0.96564263, 0.6696721, 0.49205709, 0.76099885, 0.13848515, 0.25886652, 0.8412003, 0.9281087, 0.9984723, 1.0, 0.9999524, 0.33794913], 'threshold': 0.05}, 'meta': {'name': 'TabularDrift', 'detector_type': 'offline', 'data_type': None}} 

Age -- Drift? No! -- K-S 0.007 -- p-value 0.966
Workclass -- Drift? No! -- Chi2 5.800 -- p-value 0.670
Education -- Drift? No! -- Chi2 5.413 -- p-value 0.492
Marital Status -- Drift? No! -- Chi2 1.167 -- p-value 0.761
Occupation -- Drift? No! -- Chi2 12.296 -- p-value 0.138
Relationship -- Drift? No! -- Chi2 6.520 -- p-value 0.259
Race -- Drift? No! -- Chi2 1.417 -- p-value 0.841
Sex -- Drift? No! -- Chi2 0.008 -- p-value 0.928
Capital Gain -- Drift? No! -- K-S 0.005 -- p-value 0.998
Capital Loss -- Drift? No! -- K-S 0.003 -- p-value 1.000
Hours per week 

# ChiSquareDrift

### Detecting Drift locally

In [26]:
cols = list(category_map.keys())
cat_names = [feature_names[_] for _ in list(category_map.keys())]
X_ref_cat, X_t0_cat = X_ref[:, cols], X_t0[:, cols]
X_ref_cat.shape, X_t0_cat.shape

((10000, 8), (10000, 8))

In [27]:
from alibi_detect.cd import ChiSquareDrift
cd = ChiSquareDrift(X_ref_cat, p_val=.05)
preds = cd.predict(X_t0_cat,drift_type="feature")

In [28]:
labels = ['No!', 'Yes!']
print(f"Threshold {preds['data']['threshold']}")
for f in range(cd.n_features):
    fname = cat_names[f]
    is_drift = (preds['data']['p_val'][f] < preds['data']['threshold']).astype(int)
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- Chi2 {stat_val:.3f} -- p-value {p_val:.3f}')

Threshold 0.05
Workclass -- Drift? No! -- Chi2 8.487 -- p-value 0.387
Education -- Drift? No! -- Chi2 4.753 -- p-value 0.576
Marital Status -- Drift? No! -- Chi2 3.160 -- p-value 0.368
Occupation -- Drift? No! -- Chi2 8.194 -- p-value 0.415
Relationship -- Drift? No! -- Chi2 0.485 -- p-value 0.993
Race -- Drift? No! -- Chi2 0.587 -- p-value 0.965
Sex -- Drift? No! -- Chi2 0.217 -- p-value 0.641
Country -- Drift? No! -- Chi2 9.991 -- p-value 0.441


### Detecting Drift via custom endpoint for Seldon protocol

In [29]:
from alibi_detect.utils.saving import save_detector
filepath = "alibi-detector-artifacts/detector_data_cat"
save_detector(cd, filepath)

In [33]:
%%writefile model-settings.json
{
  "name": "income-classifier-cd",
  "implementation": "mlserver_alibi_detect.AlibiDriftDetectRuntime",
  "parameters": {
    "uri": "./alibi-detector-artifacts/detector_data_cat",
    "version": "v0.1.0",
    "extra":{
      "detector_type":"alibi_detect.cd.ChiSquareDrift",
      "init_detector": false,
      "protocol": "seldon.http",
      "predict_parameters": {
        "drift_type": "feature"
      }
    }
  }
}

Overwriting model-settings.json


In [35]:
import requests

inference_request = {
    "data":{
        "ndarray": X_t0_cat.tolist()
    }
}

endpoint = "http://localhost:8080/"
response = requests.post(endpoint, json=inference_request)

In [36]:
import json
response_dict = json.loads(response.text)
print(response_dict,"\n")

labels = ['No!', 'Yes!']
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = cat_names[f]
    is_drift = response_dict['data']['is_drift'][f]
    stat_val, p_val = response_dict['data']['distance'][f], response_dict['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

{'data': {'is_drift': [0, 0, 0, 0, 0, 0, 0, 0], 'distance': [8.486513, 4.7529316, 3.159944, 8.194137, 0.48458526, 0.5865231, 0.21689056, 9.991033], 'p_val': [0.38744345, 0.5758693, 0.36761624, 0.41473988, 0.99267685, 0.9645494, 0.6414195, 0.44128036], 'threshold': 0.05}, 'meta': {'name': 'ChiSquareDrift', 'detector_type': 'offline', 'data_type': None}} 

Workclass -- Drift? No! -- K-S 8.487 -- p-value 0.387
Education -- Drift? No! -- Chi2 4.753 -- p-value 0.576
Marital Status -- Drift? No! -- Chi2 3.160 -- p-value 0.368
Occupation -- Drift? No! -- Chi2 8.194 -- p-value 0.415
Relationship -- Drift? No! -- Chi2 0.485 -- p-value 0.993
Race -- Drift? No! -- Chi2 0.587 -- p-value 0.965
Sex -- Drift? No! -- Chi2 0.217 -- p-value 0.641
Country -- Drift? No! -- Chi2 9.991 -- p-value 0.441
